# Introduction

This notebook is intended to be used as an introduction to <b>OpenCV</b> as well as other Python libraries that will help you navigate through this task. If you are unfamiliar with the task it is recommended to spend some time looking through our [github](https://github.com/WestBigDataHub/videoprivacytask) site to familiarize yourself before working through this notebook.

Any links contained in this notebook are intended to help you understand more about the highlighted topic and it is recommended to take time to read about any topics you may not understand while working through the notebook.


### Jupyter Notebooks

If you are interested in setting up a similar notebook environment locally you should follow the steps on our github site under the Tutorials section. This can be useful for downloading some of our notebooks as well as setting up a more interactive environment to work through your own implementation.

### What is OpenCV

OpenCV is a popular open-source computer vision library originally developed in C/C++ with a stable Python release being introduced in 2009. [Computer Vision](https://machinelearningmastery.com/what-is-computer-vision/) is simply a field that is concerned with developing techniques to help computers analyze and understand the content contained in an image or video through the use of algorithms. 

### Imports

In any Python project we will first need to import relevant libraries that will be used. For the purpose of this example we are going to import three libraries: 1) cv2, 2) common, 3) numpy.

1. <b>cv2</b> is simply the notation for importing the openCV library that was dicussed above
2. <b>common</b> - contains a few useful openCV functions
3. <b>numpy</b> - often abbreviated as np for simplicity, this is a Python library which adds support for large, multi-dimensional arrays and matrices as well as a collection of high-level mathematical functions to operate on these arrays. In computer vision images and video are simply collections of pixels which can be represented using arrays so this library is very useful to simplify operations on these structures.

In [ ]:
import cv2
import common
import numpy as np

# These imports are inteded to improve the notebook experience
%matplotlib inline
from matplotlib import pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

With these imports we can now display a basic image using the openCV imread function.

In [ ]:
cv2.imread('driver_face.jpg')

Notice how the output from this function is a multi-dimensional array encoding of pixel values contained in the image. We can use the below function to assign our image to a variable <b>img</b>. The image we are using in the example below comes directly from our [dataset](https://github.com/WestBigDataHub/videoprivacytask/tree/master/data) that we are using in this task.

<b>NOTE</b> - When we display images in this notebook we use plt.imshow but when working outside a jupyter notebook using opencv.imshow will open a window containing the image

In [ ]:
img = cv2.imread('driver_face.jpg')
cv2.imshow('image', img)

In [ ]:
plt.imshow(img, cmap='gray')

Notice how the colors in the above image do not look quite right. This is because openCV doesn't store images in RGB format but rather BGR format. In order to convert to the standard RGB color space we can use the following code to split our image into its various channels and then we merge those chanels in the proper order.

In [ ]:
b, g, r = cv2.split(img) #Split into various channels
merged = cv2.merge([r, g, b]) #Merge in RGB order
plt.imshow(merged)

The shorthand for doing this color conversion in openCV is shown below and is much easier to use once you understand the basic concept above

In [ ]:
opencv_merged = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(opencv_merged)

Next, we will try to get an image from a video and use our [RetinaFace](https://github.com/serengil/retinaface) (provided in our data folder on github) detection coordinates in order to crop out the face contained in a single frame of that video. For this example we pick <b>frame 30</b> of the video which roughly corresponds to the image used above.

First, we will load in the entire video and capture the image corresponding to the exact frame we need. We effectively use the VideoCapture function in order to read in the video and then we use the set function in the [VideoCapture](https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html) class to set the next frame to read. After we do this then we can call the read function to capture the proper image from that frame in the video.

In [ ]:
cap = cv2.VideoCapture('T002_ActionsShorter_mini_3239_3347_Use-Radio-or-Gadget.mp4')
total_frames = cap.get(7) # Gets the total frames from our video
cap.set(1, 30) # Sets next frame to read 
ret, frame = cap.read() # Reads next frame
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Convert frame to RGB
plt.imshow(frame_rgb)

Now that we have the desired frame we can use our RetinaFace coordinates to crop out the part of the frame containing the face. We are interested in the x, y, w, h values for each detection as these can be used to create a bounding box or capture a smaller part of the image. The detection coordiantes we have gathered are roughly the below values:<br><br>
x = 174 <br>
y = 65 <br>
w = 137 <br>
h = 194 <br>

Notice that image slicing can be done using brackets and we can combine this with our above x, y, w, h values in the form <b>image[y:y+height, x:x+width]</b>

In [ ]:
x, y, w, h = 174, 65, 137, 194
cropped = frame_rgb[y:y+h, x:x+w]
plt.imshow(cropped)

## Video Loop

Now that we have seen the above example we will look at a loop that reads in a single video and goes through each frame, capturing the corresponding face from each. This basic structure is something that you should use throughout the task for analyzing video.

In [ ]:
import pandas as pd # This library can be useful for reading in the coordinates for RetinaFace csv files

[Pandas](https://pandas.pydata.org/) is a Python library for data manipulation and analysis that offers data structures and operations for manipulating numberical tables and time series. This library may be useful outside of analyzing the RetinaFace detection coordinates but currently we are using it for analyzing the csv file containing our RetinaFace detection coordinates.

The code below reads in our csv file containing the detection coordinates and displays the first 5 rows of the table.

In [ ]:
face_detections = pd.read_csv('T002_ActionsShorter_mini_3239_3347_Use-Radio-or-Gadget.csv')
face_detections.head()

Notice how each row corresponds to a separate frame in the video. We are interested in gathering the x, y, w, h column values for each frame so that we are able to use that when analyzing the video. Here is how we can get these values for quick reference.

In [ ]:
coordinates = face_detections[['x', 'y', 'w', 'h']].round(0).astype(int)
coordinates.head()

This is the basic structure of a loop in opencv that iterates through the video and captures the face at each frame

In [ ]:
face_detections = pd.read_csv('T002_ActionsShorter_mini_3239_3347_Use-Radio-or-Gadget.csv')
coordinates = face_detections[['x', 'y', 'w', 'h']]
coordinates = coordinates.round(0).astype(int) # Rounds the detection coordinates and converts to integers

cap = cv2.VideoCapture('T002_ActionsShorter_mini_3239_3347_Use-Radio-or-Gadget.mp4')
success, img = cap.read()
frame_number = 0
while success:
    x, y, w, h = coordinates.iloc[0] # Gets the coordinates and assign values to x, y, w, h
    face = img[y:y+h, x:x+w]
    # Do something with the face image
    success, img = cap.read() # Iterate to next frame
    frame_number += 1

## More Materials

If you found this tutorial helpful and want to learn more about openCV usage [this](https://medium.com/analytics-vidhya/introduction-to-opencv-cc771730577a) link will be helpful. It contains a collection of 4 notebooks (the material in the first one has been covered here) that are useful in learning the basics of image processing, features in computer vision and cascade classification. 

If you are interested in some more openCV tutorials these can be found [here](https://docs.opencv.org/4.5.2/d6/d00/tutorial_py_root.html). These tutorials have a broader range in topics and can be a good reference when learning what openCV can do.